In [ ]:
import pandas as pd
import boto3

In [ ]:
df = pd.read_csv('googleplaystore.csv')
df.info()

In [ ]:
# Remove no value rows
df.dropna(inplace=True)
# Features to be used
features = ['Rating', 'Reviews', 'Size', 'Installs', 'Type','Category', 'Price', 'Content Rating', 'Genres']
df = df.loc[:,features]
df.head()

In [82]:
# Function to convert categorical data
def convert_category_like(s):
    temp_dict = {}
    for i, k in enumerate(s.unique()):
        temp_dict[k]= i
    return s.map(temp_dict).astype(int)

In [83]:
# Convert categorical data
df['Category_c'] = convert_category_like(df['Category'])
df['Content Rating'] = convert_category_like(df['Content Rating'])
df['Genres_c'] = convert_category_like(df['Genres'])

In [84]:
# Convert string to numeric
df['Reviews'] = df['Reviews'].astype('int64')

# Function to convert size
def convert_size(s):
    if s[-1] == 'M':
        ans = float(s[:-1])*1000000
    elif s[-1] == 'k':
        ans = float(s[:-1])*1000
    else:
        ans = None
    return ans
df['Size'] = df['Size'].map(convert_size)
# Some size value is "Varies with device", we fill those value with mean groupby categories
df['Size'] = df.groupby('Category_c')['Size'].transform(lambda x: x.fillna(round(x.mean())))

In [85]:
# Convert Installs to numeric
df['Installs'] = df['Installs'].map(lambda x: int(x[:-1].replace(',','')), )
# Convert Type to binary
df['Type'] = df['Type'].map(lambda x: 0 if x == 'Free' else 1)
# Convert Price to numeric
df['Price'] = df['Price'].map(lambda x: 0 if x == '0' else float(x[1:]))